# 🎬 Wan 2.1 Video Studio - Colab Launcher

이 노트북은 Google Colab에서 Wan 2.1 Video Studio를 실행합니다.
T4 GPU 이상이 필요합니다.

**실행 방법**: 각 셀을 순서대로 실행하세요.

In [ ]:
# GPU 확인
import torch
assert torch.cuda.is_available(), "GPU가 필요합니다. 런타임 → 런타임 유형 변경 → GPU를 선택하세요."
gpu_name = torch.cuda.get_device_name(0)
vram_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
print(f"✅ GPU: {gpu_name}")
print(f"✅ VRAM: {vram_gb:.1f} GB")
if vram_gb < 14:
    print("⚠️ VRAM이 14GB 미만입니다. 480P 모드만 안정적으로 사용 가능합니다.")

In [ ]:
# 앱 코드 설치 + 환경변수 로드
import os
APP_DIR = "/content/image-video-studio"

if not os.path.exists(APP_DIR):
    !git clone https://github.com/hyunlord/image-video-studio.git {APP_DIR}
else:
    !git -C {APP_DIR} pull --ff-only
    print("앱 코드가 이미 존재합니다. (최신 pull 완료)")

%cd {APP_DIR}
!pip install -q -r requirements.txt

# .env 파일이 있으면 환경변수 로드
from pathlib import Path
env_file = Path(APP_DIR) / ".env"
if env_file.exists():
    for line in env_file.read_text().splitlines():
        line = line.strip()
        if line and not line.startswith("#") and "=" in line:
            key, _, value = line.partition("=")
            if value.strip():
                os.environ[key.strip()] = value.strip()
    print("✅ .env 환경변수 로드 완료")
else:
    print("💡 .env 파일 없음 (선택사항: .env.sample 참고하여 생성)")
print("✅ 앱 의존성 설치 완료")

In [ ]:
# Wan 2.1 모델 설치 (~28GB, 약 10-15분 소요)
import os

WAN21_DIR = "/content/Wan2.1"
CKPT_DIR = f"{WAN21_DIR}/ckpts/FLF2V-14B-720P"

# Clone Wan 2.1
if not os.path.exists(f"{WAN21_DIR}/.git"):
    !git clone --depth 1 https://github.com/Wan-Video/Wan2.1.git {WAN21_DIR}
    !pip install -q -r {WAN21_DIR}/requirements.txt
    # Wan 2.1이 numpy를 다운그레이드하므로 호환 버전으로 재설치
    !pip install -q "numpy>=2.0" --force-reinstall
    print("✅ Wan 2.1 설치 완료")
else:
    print("✅ Wan 2.1 이미 설치됨")

# Download FLF2V-14B model
if not os.path.exists(CKPT_DIR) or len(os.listdir(CKPT_DIR)) == 0:
    print("📥 FLF2V-14B 모델 다운로드 중... (~28GB, 10-15분 소요)")
    from huggingface_hub import snapshot_download

    hf_token = os.environ.get("HF_TOKEN") or None
    snapshot_download(
        repo_id="Wan-AI/Wan2.1-FLF2V-14B-720P",
        local_dir=CKPT_DIR,
        token=hf_token,
    )
    print("✅ 모델 다운로드 완료")
else:
    print("✅ FLF2V-14B 모델 이미 다운로드됨")

In [ ]:
# 후처리 도구 설치
import os

# Real-ESRGAN (basicsr 포함 - CodeFormer보다 먼저 설치)
try:
    import realesrgan
    print("✅ Real-ESRGAN 이미 설치됨")
except ImportError:
    !pip install -q realesrgan
    print("✅ Real-ESRGAN 설치 완료")

# CodeFormer
CODEFORMER_DIR = "/content/CodeFormer"
if not os.path.exists(f"{CODEFORMER_DIR}/.git"):
    !git clone --depth 1 https://github.com/sczhou/CodeFormer.git {CODEFORMER_DIR}
    !pip install -q -r {CODEFORMER_DIR}/requirements.txt
    %cd {CODEFORMER_DIR}
    !python scripts/download_pretrained_models.py CodeFormer
    %cd /content/image-video-studio
    print("✅ CodeFormer 설치 완료")
else:
    print("✅ CodeFormer 이미 설치됨")

# RIFE
RIFE_DIR = "/content/RIFE"
if not os.path.exists(f"{RIFE_DIR}/.git"):
    !git clone --depth 1 https://github.com/hzwer/ECCV2022-RIFE.git {RIFE_DIR}
    print("✅ RIFE 설치 완료")
else:
    print("✅ RIFE 이미 설치됨")

print("\n✅ 모든 후처리 도구 설치 완료")

In [ ]:
# 서버 시작 + 터널링
import os, sys, threading

# Set environment variables
os.environ["WAN21_DIR"] = "/content/Wan2.1"
os.environ["CODEFORMER_DIR"] = "/content/CodeFormer"
os.environ["RIFE_DIR"] = "/content/RIFE"
os.environ["MODEL_CACHE_DIR"] = "/content/Wan2.1/ckpts/FLF2V-14B-720P"

# Add app to path
APP_DIR = "/content/image-video-studio"
if APP_DIR not in sys.path:
    sys.path.insert(0, APP_DIR)

# Start ngrok tunnel
try:
    from pyngrok import ngrok
except ImportError:
    !pip install -q pyngrok
    from pyngrok import ngrok
ngrok.kill()

ngrok_token = os.environ.get("NGROK_AUTH_TOKEN")
if ngrok_token:
    ngrok.set_auth_token(ngrok_token)

tunnel = ngrok.connect(8000)
public_url = tunnel.public_url
print(f"🌐 Public URL: {public_url}")

# Start uvicorn in background thread
import uvicorn
from backend.app import app

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

print(f"\n{'='*50}")
print(f"🎬 Wan 2.1 Video Studio 실행 중!")
print(f"🌐 접속 URL: {public_url}")
print(f"{'='*50}")
print(f"\n⚠️ 이 셀을 중단하면 서버가 종료됩니다.")
print(f"💡 위 URL을 클릭하여 웹 브라우저에서 접속하세요.")